In [1]:
#import the needed packages, one needed once per Workingstation
#import Pkg; Pkg.add("DecisionTree")
#import Pkg; Pkg.add("CSV")
#import Pkg; Pkg.add("DataFrames")
#import Pkg; Pkg.add("TypedTables") #needed for the table at the end
#import Pkg; Pkg.add("JLD2")        #needed to save the models
#import Pkg; Pkg.add("TickTock")
# Activating the packages needed, allways needed
using DecisionTree
using CSV, DataFrames
using TypedTables
using JLD2
using Random
import DataFrames, XLSX
using CSV, Glob, DataFrames
# use TickTock to time the steps ( tick() at the beginning, tock() at the end)
using TickTock

In [2]:
# Location of the training dataset with all the combined training data
# there can be multiple folders in the directory but only one .csv file
TrainingData = "E://BA//machine learning//comp_datasets//training//Without_FL"
# to replace all \ windows uses press Esc + f, enter \ in find and replace with //
# last folder (eg t0) should not be named longer then 
# Data for L t24
path1 = "E://BA//machine learning//accuracy//without_FL//L//t0" 
# Data for U t0
path2 ="E://BA//machine learning//accuracy//without_FL//U//t0"
# Data for U t24
path3 = "E://BA//machine learning//accuracy//without_FL//L//t8" 
# Data for U t8
path4 = "E://BA//machine learning//accuracy//without_FL//U//t8"
# Data for L t8
path5 = "E://BA//machine learning//accuracy//without_FL//L//t24" 
# Data for U t24
path6 = "E://BA//machine learning//accuracy//without_FL//U//t24"
#comibines all paths above for automatisation
paths = (path1,path2,path3,path4,path5,path6)
# Names of the bacteria
NameBac1 = "BS"
NameBac2 = "EC"
# number of replicates 
# Where to save an Excel file with the result e.g. E://BA//EX2//t24utest//t24.xlsx
here = "E://BA//machine learning//Results//test01" 
# Name of the result files : Results name of the last two file from path
# e.g. Results Lt0.xlsx

"E://BA//machine learning//Results//test01"

In [3]:
tick()
Tfile=glob("*.csv", TrainingData) # to load all CSVs from a folder (* means arbitrary pattern)
# randomizes the test dataset for better accuracy
dfTrain = DataFrame(CSV.File(Tfile))
rdfTrain = dfTrain[shuffle(1:size(dfTrain, 1)),:]

# Define whats a features and whats a label (Name of the Bactiria)
features = float.(Matrix(rdfTrain[:, 1:18]))
labels   = string.(rdfTrain[:, 19])

# Building the forest using 2 random features, 1000 trees, 0.5 portion of samples per tree, and a maximum tree depth of 40
model = build_forest(labels, features, 2, 1000, 0.2, 1000)
# to save this model use
@save "BSmKate_EcoliGFP_t0-t24,0.1.jld2" model

# if you have a model load use  
#@load("BSmKate_EcoliGFP_t0-t24.jld2")
tock()

┌ Info:  started timer at: 2023-08-14T14:42:26.960
└ @ TickTock C:\Users\Nikolas\.julia\packages\TickTock\fGILW\src\TickTock.jl:54
┌ Info:          467.5730788s: 7 minutes, 47 seconds, 573 milliseconds
└ @ TickTock C:\Users\Nikolas\.julia\packages\TickTock\fGILW\src\TickTock.jl:62


In [4]:
suffix = ".xlsx" # used later on to create a file as an .xlsx file
dd = "//"
n = length(paths)
tick()
# classifiacation of all the files in each path
for i in 1:n
    C = DataFrame([[],[]], [NameBac1, NameBac2]) # creates an empty data frame 
    p = paths[i] # defines p as the path for where the data is eg. for t0 
    re = readdir(p) # reads directory (all files in p)
    r = length(re) # counts numer of folders read, needed to repeat the next loop according
    for i in 1:r
        files = glob("*.csv", p.*dd.*re[i]) # reads in all the files from re
        y = length(files) # counts numer of files, needed to repeat the next loop according
        this =re[i] # defines this as the name of i file of re
        DFs = DataFrame.(CSV.File.(files)) # creates the data frames form files
        for i in 1:y
            if i == 1 # if i == 1 push the name of the file as 1. row in the empty data frame
                push!(C,[this,""]) 
            end
            test = float.(Matrix(DFs[i][:, 1:18])) # creates a matrix from the data frame 
            frame = apply_forest(model, test[:, 1:18];) # applies the model to the data frame 
            TestArray = findall( x -> x == NameBac1, frame) # lists all NameBac1 in the frame 
                                                            # created after applying the model
            Bac1c = (length(TestArray)/length(frame)) # counts the number NameBac1 in the frame
            Bac2c = (1- Bac1c) # calculates how much NameBac2 is left
            push!(C,[Bac1c,Bac2c]) # pushes the numbers in c (the empty data frame)
        end
    end
   # println(this,"
   #     ",C) # prints the name of the frame and the frame
    a = split(paths[i],dd) # splits the path where the file is at ever //
    # creates a result name = Results and the last to parts of the path name e.g. Resultst0L
    results = "Results "*(a[length(a)])*(a[(length(a)-1)])
    # writes the xlsx file here//results.xlsx
    XLSX.writetable(here.*dd.*results.*suffix,("REPORT") => C) 
end 
tock()

┌ Info:  started timer at: 2023-08-14T14:50:14.649
└ @ TickTock C:\Users\Nikolas\.julia\packages\TickTock\fGILW\src\TickTock.jl:54
┌ Info:         2462.0003277s: 41 minutes, 2 seconds
└ @ TickTock C:\Users\Nikolas\.julia\packages\TickTock\fGILW\src\TickTock.jl:62
